In [1]:
import numpy as np
import pandas as pd
import re
bank_txt = spark.sparkContext.textFile("/data/examples/bank.csv")

bank_df = bank_txt.map(lambda s: re.sub(r'(?:^\"|\"$)', '', s))\
    .map(lambda s: re.split('\"?;\"?', s))\
    .filter(lambda s: s[0] != "age")\
    .map(lambda s: [int(s[0])] + s[1:4] + [int(s[5])]).toDF(['age', 'job', 'marital', 'education', 'balance'])
bank_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- balance: long (nullable = true)



In [2]:
bank_df2 = spark.read.csv("/data/examples/bank.csv",
                         inferSchema=True, header=True, sep=";", quote='"')
bank_df2.toPandas().head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
bank_df = bank_df2.select(['age', 'job', 'marital', 'education', 'balance'])
bank_df.limit(5).toPandas()

,age,job,marital,education,balance
0,30,unemployed,married,primary,1787
1,33,services,married,secondary,4789
2,35,management,single,tertiary,1350
3,30,management,married,tertiary,1476
4,59,blue-collar,married,secondary,0


In [4]:
bank_df.createOrReplaceTempView('bank_view')

In [5]:
em_pdf = spark.sql('''SELECT education, marital, count(*) AS count FROM bank_view
WHERE education != 'unknown' GROUP BY education,marital''').toPandas()
em_pdf.head(3)

Hive Session ID = 80007668-8cdf-46a9-9ad3-07cc37e8ca8e


,education,marital,count
0,primary,single,73
1,tertiary,single,468
2,secondary,divorced,270


In [6]:
pdf = em_pdf.pivot(index='education', columns='marital', values='count')
pdf.head(3)

marital,divorced,married,single
education,,,
primary,79,526,73
secondary,270,1427,609
tertiary,155,727,468


In [ ]:
import matplotlib.pyplot as plt
pdf.plot(kind='bar', stacked=True)
plt.show()

In [ ]:
df = spark.read.csv("/data/examples/bank.csv", header=True, sep=';', 
                      mode="DROPMALFORMED", inferSchema=True)
df.write.mode("overwrite").saveAsTable("default.bank");
df.printSchema()

In [ ]:
pd.DataFrame(spark.catalog.listTables("default"))

In [ ]:
bank_df = spark.read.table("bank")
bank_jobs = bank_df.groupBy("job").count()
bank_jobs.createOrReplaceTempView("bank_jobs")
jobs_pd = spark.sql("select * from bank_jobs order by count desc limit 10").toPandas()
jobs_pd.head()

In [ ]:
jobs_pd.plot(kind='bar', x='job', y='count', stacked=True)
plt.show()